In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy

In [2]:
# def function to scrape internsg 
def internSgScrapper(pages=39, output_file_path='internSG_jobs.csv'):
    # get all pages first 
    list_of_all_pages = []
    for i in range(1,pages):
        url = f'https://www.internsg.com/jobs/{i}/?f_p=107&f_i&filter_s#isg-top'
        list_of_all_pages.append(url)

    # get title of all jobs 
    data = []
    for url in list_of_all_pages:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        divs = soup.find_all('div', class_='ast-col-lg-3')
        for div in divs:
            # Try to find an <a> tag within the <div>
            a_tag = div.find('a')
            # If an <a> tag is found and it has a 'href' attribute
            if a_tag and 'href' in a_tag.attrs:
                # Extract the URL and the job title
                url = a_tag['href']
                job_title = a_tag.get_text().strip()
                # Append the data to the list
                date = 'Not Available'
                # Try to find the next sibling 'div' which might contain the date
                date_div = div.find_next_sibling('div', class_='ast-col-lg-1')
                if date_div:
                    date_span = date_div.find('span', class_='text-monospace')
                    if date_span:
                        date = date_span.get_text().strip()
                data.append({'URL': url, 'Job Title': job_title, 'Date':date})

    title_url_df = pd.DataFrame(data)
    col_names = ['Company','Designation','Date Listed','Job Type','Job Period','Profession',
             'Industry','Location Name','Allowance / Remuneration','Company Profile',
             'Job Description']
    jobs_info = []
    for url in title_url_df['URL']:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        res_dict = dict.fromkeys(col_names, '')
        res_dict["url"] = url
        for col_name in col_names:
            # Find the div that contains the column name
            col_div = soup.find('div', text=col_name, class_='font-weight-bold')
            if col_div:
                # The actual data is in the next sibling of the parent of col_div
                next_div = col_div.find_next_sibling()
                if next_div:
                    # Extract the text and store it in the dictionary
                    for span in next_div.find_all('span'):
                        span.decompose()
                    res_dict[col_name] = next_div.get_text(strip=True)
        jobs_info.append(res_dict)
    
    # return results 
    internSG_jobs = pd.DataFrame(jobs_info)
    internSG_jobs.to_csv(output_file_path,index=False)
    return internSG_jobs

In [3]:
internSgScrapper(40)

,Company,Designation,Date Listed,Job Type,Job Period,Profession,Industry,Location Name,Allowance / Remuneration,Company Profile,Job Description,url
0,Will Group Asia Pacific Pte Ltd,IT Intern,25 Apr 2024,"Entry Level / Junior Executive, Experienced / ...","Immediate Start, For At Least 3 Months",IT / Information Technology,Human Resources,"Anson Road, Singapore","$1,500 - 3,000 monthly",Established Work force solution company with r...,Key ResponsibilitiesProject Management: Assist...,https://www.internsg.com/job/will-group-asia-p...
1,The People Lab Pte Ltd,UI / UX Front End Designer,25 Apr 2024,Entry Level / Junior Executive,"From May 2024, For At Least 1 Month",IT / Information Technology,Human Resources,Singapore,"$4,500 - 5,000 monthly","At THE PEOPLE LAB, we understand the pain poin...",Responsibilities:Developing intuitive and enga...,https://www.internsg.com/job/the-people-lab-pt...
2,Holmusk (KKT Technology Pte Ltd),Data Science Intern,24 Apr 2024,Entry Level / Junior Executive,"From May 2024, For At Least 3 Months",IT / Information Technology,Healthcare / Fitness / Sports,"71 Ayer Rajah Crescent, Singapore","$1,500 monthly",At Holmusk we leverage technology & data scien...,ResponsibilitiesReview and test Holmusk specif...,https://www.internsg.com/job/holmusk-kkt-techn...
3,NCS Group,IT Support Engineer,24 Apr 2024,Entry Level / Junior Executive,Immediate Start - 31 Mar 2026,IT / Information Technology,Computer and IT,Singapore,"$1,900 - 2,400 monthly","NCS is a leading technology services firm, ope...",This position provides the desktop support for...,https://www.internsg.com/job/ncs-group-it-supp...
4,Univers,Security Analyst Intern,24 Apr 2024,Entry Level / Junior Executive,From May 2024 - May 2025,IT / Information Technology,Computer and IT,"1 HarbourFront Avenue, Keppel Bay Tower, Singa...","$2,300 monthly",Univers provides the world’s most comprehensiv...,SOC Intern Security Analyst InternResponsibili...,https://www.internsg.com/job/univers-security-...
...,...,...,...,...,...,...,...,...,...,...,...,...
769,ACP Group,Mobile Developer Intern,18 May 2023,Entry Level / Junior Executive,"From 24 Jul 2023, For At Least 4 Months",IT / Information Technology,Computer and IT,"Ang Mo Kio Street 62, Link@AMK, Singapore","$800 - 1,000 monthly",About ACPACP is the leading provider of Learni...,Roles & ResponsibilitiesAssist in the developi...,https://www.internsg.com/job/acp-group-mobile-...
770,Snaphunt Pte Ltd,Technical Support Engineer,18 May 2023,Entry Level / Junior Executive,"Immediate Start, Permanent",IT / Information Technology,Computer and IT,Singapore,"$3,000 - 3,500 monthly",Our client is an IT Services and recruiting co...,Work within a company with a solid track recor...,https://www.internsg.com/job/snaphunt-pte-ltd-...
771,Mitsubishi Corporation RtM International Pte Ltd,Digital Transformation Intern,16 May 2023,Entry Level / Junior Executive,"Flexible Start, For At Least 6 Months",IT / Information Technology,Wholesale Trade,"Temasek Avenue, Centennial Tower, Singapore","$800 - 1,200 monthly",Mitsubishi Corporation RtM International (RtMI...,Purpose of roleThe intern would support the Di...,https://www.internsg.com/job/mitsubishi-corpor...
772,Snaphunt Pte Ltd,Junior Network Engineer,15 May 2023,Entry Level / Junior Executive,"Immediate Start, Permanent",IT / Information Technology,Computer and IT,Singapore,"$2,500 - 3,500 monthly",Our client is part of a group of publishing co...,Join a team of world class IT professionalsCom...,https://www.internsg.com/job/snaphunt-pte-ltd-...
